# Claim Feature Modelling

This notebook looks at some object models for patent claim features.

We have two main types of patent claim: apparatus and method.  

The "apparatus" claim type includes system and kit claims. This claim covers a physical object or set of objects.

The "method" claim type covers a method or process. It consists of a sequence of steps.

A physical object has multiple parts. These may feature in the "apparatus" patent claim. The object or its parts may have limitations that define the structure of the object or part. An object or its parts are defined by nouns, whereas the interaction between parts is defined by verbs (commonly in the past tense). We can refer to an object, an object part, or a system or set of objects as an "entity".

Steps of a "method" claim are verbs using the present participle.

Our "apparatus" claim may be represented as a series of nested graphs. 

A single graph may be represented as:
```
graph = { "a" : ["c"],
          "b" : ["c", "e"],
          "c" : ["a", "b", "d", "e"],
          "d" : ["c"],
          "e" : ["c", "b"],
          "f" : []
        }
```
We would have something similar, but each node would be an "entity" representation. 

In [1]:
c = [1, 2, 3]
c += [2]
print(c)
c.pop(2)
print(c)

[1, 2, 3, 2]
[1, 2, 2]


In [2]:
# Python graph class tutorial- http://www.python-course.eu/graphs_python.php
class StringFeature:
    """ Abstract object for a phrase or word.
     
     Attributes:
        headpos - the head part of speech, e.g. noun or verb
        tokens - tuple of (string, POS_tag)        
    """
    def __init__(self, tokens):
        """ Initialise object.
        
        tokens - list of tuples of (string, POS_tag)
        """
        self.tokens = tokens
    
    def __repr__(self):
        return " ".join([word for word, _ in self.tokens])
    
    @property
    def stringphrase(self):
        return self.__repr__()
    
class ClaimFeature(StringFeature):
    """ Abstract object for instantiating entities and steps. 
    
    Attributes:
        ref_num - int representing associated reference number (maybe a list?)
        parent (? - or get from navigating children)
        children
        limitations
        
    
    """
    def __init__(self, tokens):
        """ Initialise object.
        
        tokens - list of tuples of (string, POS_tag)
        """
        super().__init__(tokens) 
        self.children = list()
        self.limitations = list()
       
    def add_child(self, child):
        """ Add a child entity.
        
        child: an object of the same class
        """
        # check same instance type
        assert isinstance(child, type(self))
        self.children.append(child)
        return self.children
    
    def remove_child(self, child):
        """ Remove a child entity.
        
        child: a child object to remove
        """
        self.children.pop(child)
        return self.children
    
    def add_limitation(self, limitation):
        """ Add a limitation.
        
        limitation: a Limitation object
        """
        assert isinstance(limitation, Limitation)
        self.limitations.append(limitation)
        return self.limitations
    
    def remove_limitation(self, limitation):
        """ Remove a limitation.
        
       limitation: a Limitation object
        """
        self.limitations.pop(limitation)
        return self.limitations
    
    def prettyprint(self, object_str_single, object_str_plural):
        """ Pretty print a representation of feature with
        children and limitations.
        
        object_str_single: string to call feature instantiation 
        object_str_plural: string to call feature instantiation (plural)
        """
        
        print("{0}: {1}\n".format(object_str_single, self.__repr__()))
        
        print("Limitations:\n")
        for i, limitation in enumerate(self.limitations):
            print("\t{0} - {1}".format(i, limitation.__repr__()))
        
        print("Child {0}:\n".format(object_str_plural))
        for i, child in enumerate(self.children):
            print("\t{0} - {1}".format(i, child.__repr__()))
    
    
class Entity(ClaimFeature):
    """ Object to represent an entity in a claim. 
    
    Attributes:
        see ClaimFeature
        headpos = headnoun
    
    """
    def prettyprint(self):
        super().prettyprint("Entity", "Entities")
        

class Step(ClaimFeature):
    """ Object to represent a step in a claim. 
    
    Attributes:
        see ClaimFeature
        headpos = headnoun
        
    """
    def prettyprint(self):
        super().prettyprint("Step", "Steps")
        
        
class Limitation(StringFeature):
    """ Object to represent a limitation applied to an entity or method step.
    
    Attributes:
        see StringFeature
    
    """
    pass

In [3]:
class Claim:
    """ Abstract object to represent a claim. 
    
    Attributes:
        features
    """
    def __init__(self, features=[]):
        """ Initialise object.
        
        features - list of instantiated ClaimFeature objects
        """
        self.features = features

class MethodClaim(Claim):
    """ Object to represent an apparatus claim. 
    
    Attributes:
        steps
    """
    def __init__(self, steps=[]):
        """ Initialise object.
        
        steps - list of Step objects
        """
        super().__init__(steps) 
        
    @property
    def steps(self):
        return self.features

class ApparatusClaim(Claim):
    """ Object to represent an apparatus claim. 
    
    Attributes:
        entities  
    """
    def __init__(self, entities=[]):
        """ Initialise object.
        
        entities - list of Entity objects
        """
        super().__init__(entities) 
    
    @property
    def entities(self):
        return self.features

In [4]:
a = ApparatusClaim()
e = Entity([("an", "DET"), ("apparatus", "NOUN")])

In [5]:
e

an apparatus

In [6]:
e.add_child(e)

[an apparatus]

In [7]:
e.prettyprint()

Entity: an apparatus

Limitations:

Child Entities:

	0 - an apparatus


In [8]:
a.features = [e, e]

In [9]:
a.entities

[an apparatus, an apparatus]

We could do with the init and add limitation working with strings that get converted behind the scenes?  

But POS won't be accurate on string portions. But when we are drafting we will not have POS.

Add child could take a string and convert into an entity/step behind the scenes?

For claim printing we want to generate a string representation from the parts.